# Model Deployment with Flask

In this notebook, we’ll learn how to deploy a trained Machine Learning model using **Flask**, a lightweight web framework in Python. Flask helps convert your ML models into web APIs that can interact with real-world applications such as websites, apps, or other systems.

### 🔍 Objectives:
- Understand how to serve ML models via REST API.
- Build a simple Flask app to make predictions.
- Test model inference locally using `requests` or `curl`.


## 1️⃣ Import Required Libraries

In [ ]:
import numpy as np
import pickle
from flask import Flask, request, jsonify

## 2️⃣ Load the Trained Model

Assume you already have a model trained and saved using `pickle` or `joblib`. We'll load it here for inference.

In [ ]:
# Example: Load a sample scikit-learn model
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

## 3️⃣ Create a Flask App

In [ ]:
app = Flask(__name__)

@app.route('/')
def home():
    return 'Welcome to the ML Model API!'

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = np.array(data['features']).reshape(1, -1)
    prediction = model.predict(features)
    return jsonify({'prediction': prediction.tolist()})

## 4️⃣ Run the Flask Server

Once you save this script as `app.py`, you can run it using:

```bash
python app.py
```

The API will be available at `http://127.0.0.1:5000/`.


In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

## 5️⃣ Test the API

Use Python’s `requests` library or `curl` command to send a test request.

**Example (Python):**
```python
import requests
url = 'http://127.0.0.1:5000/predict'
payload = {'features': [5.1, 3.5, 1.4, 0.2]}
response = requests.post(url, json=payload)
print(response.json())
```

**Example (curl):**
```bash
curl -X POST -H "Content-Type: application/json" -d '{"features":[5.1,3.5,1.4,0.2]}' http://127.0.0.1:5000/predict
```


## ✅ Summary

- Flask is used to wrap ML models as REST APIs.
- You can deploy this API locally, on cloud servers, or platforms like Heroku and AWS.
- For production, consider tools like **FastAPI**, **Docker**, and **Gunicorn** for scaling.

**Next Steps:** Try deploying your model using FastAPI or Streamlit!